# Roulette Arbitrage

I found an online casino which offered a deal where it would double your roulette stake for you, but you must play 25 spins before you could withdraw your investment plus winnings (or whatever is left). I wanted to investigate if an arbitrage opportunity existed where I could guarantee finishing with a profit after 25 spins. 

I focus only on betting on the colours red/black/green. I am not aware of any more complex strategies that lower the house edge so I thought it best to keep it simple. However, given that roulette can only be beaten by identifying skewed wheels or using a wearable computer to track spin trajectory I don't see any use for advanced strategies. I am more interested in seeing if I can arb this deal offered by the casino.

## Assumptions

There are several inherent assumptions in the below analysis.

 - It is assumed that the casino and the game itself is fair and that numbers are chosen at random from a uniform distribution.
 - The deal suggests that each wager need only be the minimum of initial capital and capital remaining. If you lose some of your stake you do not need to top the account up. Similarly, you can hold out any profits you win and continue betting only the amount of initial investment.
 - The game under consideration is European roulette which has 37 slots, 18 red, 18 black and 1 green. The same analysis would hold for American roulette but the probabilities and payoffs would change.

## Expected Value

Suppose we have a total stake of \\$y. We place a wager of \\$x on both red and black and the remaining $(y - 2x) on green (where 2x <= y). The expected value of this bet is:

\begin{eqnarray}
\mathbf{EV} &=& \frac{36}{37}(-y + 2x) + \frac{1}{37}(35(y - 2x) - 2x) \\
            &=& \frac{36}{37}(-y + 2x) + \frac{1}{37}(35y - 72x) \\
            &=& \frac{72x}{37} - \frac{36y}{37} + \frac{35y}{37} - \frac{72x}{37} \\
            &=& -\frac{y}{37}
\end{eqnarray}

The expected value of one spin of roulette is independent of the amount we wager on red/black/green. On average we can expect to lose 1/37, approximately 2.7%, of our total stake with each spin. This is due to the house edge which is where a casino makes its profit. The odds of the ball landing on any individual number is 1/37, however the payout is only 35:1 or 36 times your stake. Therefore, the casino is taking 1/37 of the capital remaining every spin. 

\begin{align}
(1-\frac{1}{37})^{25} = 0.5041
\end{align}

If we only lose the house edge each spin after 25 spins 50.41% of the total stake remains. As we have only contributed 50% of the capital used for the strategy this results in an edge of 0.82%.

## Monte Carlo Simulation

### Hedged Strategy

To capture this edge we need a strategy that is perfectly hedged against any outcome and loses exactly 1/37 of capital with each spin. To calculate how to structure the wager for each spin, assume again we have \\$y of capital and we will wager \\$g on green and \\$rb on both red and black. For this to be perfectly hedged we also need that the payoff of this strategy is the same regardless of which colour the ball lands on. This leads to the following system of equations:

\begin{eqnarray}
2rb + g &=& y \\
2rb &=& 36g
\end{eqnarray}

Solving simultaneously we obtain:

\begin{eqnarray}
g &=& \frac{y}{37},     \
rb &=& \frac{18y}{37}
\end{eqnarray}

Staking 1/37 of capital on green and splitting the remainder evenly between red and black results in constraining the loss on each spin to the house edge. Each iteration of the monte carlo decays in the same way always finishing with a 0.82% profit after 25 spins.

<table><tr>
<td> <img src="Path_hedged.png" alt="Drawing" style="width: 300px;"/> </td>
<td> <img src="Dist_hedged.png" alt="Drawing" style="width: 300px;"/> </td>
    </tr></table>

I then explored the effect of altering the betting strategy by assigning proportionally more or less capital to green.

### Short Green Strategy

This strategy involves splitting the stake evenly betwen red and black, essentially betting against the occurrence of green over the course of 25 spins. If the ball does not land on green during the 25 spins, I keep the entire stake. However, if it lands on green at all I lose everything. Over the course of 25 spins there is a 50.41% chance (see above workings) that the ball will not land on green giving this strategy a slight positive edge over many monte carlo iterations. This probabilistic relationship is evident in the distribution graph below.

<table><tr>
<td> <img src="Path_shortgreen.png" alt="Drawing" style="width: 300px;"/> </td>
<td> <img src="Dist_shortgreen.png" alt="Drawing" style="width: 300px;"/> </td>
    </tr></table>

Assuming a total initial stake of $200, the expected value of this strategy can be calculated as:

\begin{eqnarray}
\mathbf{EV} &=& 200(0.5041) + 0(1-0.5041) \\
            &=& 100.82
\end{eqnarray}

While each individual iteration is essentially an all-or-nothing coin flip, in the long run this strategy has an edge of 0.82%, identical to that of the hedging strategy. This particular simulation had an edge of 1.1% after 10000 iterations. As the number of iterations in the simulation increases this would converge to 0.82% as the law of large numbers erodes the effect of randomness.

While both the short strategy and hedging strategy offer the same long-run return, the distribution of these returns are very different. With hedging there is no variance in returns and 0.82% can be earned after every 25 spins. However, with the short strategy you earn a 100% profit approximately half the time and lose 100% the rest of the time resulting in a variance of approximately 10000.

The risk-reward trade-off clearly favours the hedging strategy as it offers the same long-run return without any of the risk.

### Long Green Strategy

I also explored the alternate extremity of betting the entire stake on green. While this seems like the type punt that leads to large gambling losses, it produced some interesting results due to the way the casino offered this deal. A key element of this strategy is that when you are in the money so have grown capital above the initial investment, you only have to keep betting the same initial investment rather than the total amount in your account. For example, suppose we invest \\$100 of our own money resulting in overall capital of \\$200. If the ball lands on green initially our capital jumps to \\$7200, but we still only need to wager \\$200 each spin allowing us to keep our profits.

This strategy relies on the ball landing on green on the first spin which occurs approximately 2.7% of the time. Once this has happened capital then decays linearly by a function of the initial investment while occasionally taking discrete jumps when green is landed on again. 

<table><tr>
<td> <img src="Path_longgreen.png" alt="Drawing" style="width: 300px;"/> </td>
<td> <img src="Dist_longgreen.png" alt="Drawing" style="width: 300px;"/> </td>
    </tr></table>

The distribution of the returns shows that approximately 97% of simulations result in all capital being lost. However, the magnitude of the returns when the strategy makes money far outweight the losses in terms of expected return resulting in an edge of 90%.

While this appears to be a significant edge, in reality it is not a feasible strategy. It is heavily subject to randomness and thus requires an extremely large amount of spins for it to become profitable in the long-run. Entering this 25 spin bet hundreds or thousands of times is extremely time inefficient and would also likely be restricted by the casino itself. Moreover, it would require a large capital backing given that your entire stake will be lost approximately 97% of the time.

### Conclusion

An arbitrage opportunity exists which guarantees 0.82% return after 25 spins. This strategy is perfectly hedged against any outcome of each individual spin, constraining the loss each spin to the house edge. Altering the strategy to be more heavily weighted towards certain outcomes did not tilt the edge in our favour. It introduced much more variance in returns or suggested logistically infeasible strategies.

In [ ]:
import pandas as pd
import random
import math
import statistics
import matplotlib.pyplot as plt

investment = 100
total_stake = 2*investment


def expected_value(investment,stake):
    current_value = stake
    for spin in range(1,26):
        current_value = current_value*(36/37)
        
    edge = ((current_value - investment)/investment)*100
    return str(round(edge,2)) + "%"

def payoffs(num,rb_stake,g_stake):
    payoff = 0
    if num == 0:
        payoff = 36*g_stake
    else:
        payoff = 2*rb_stake
    return payoff

def simulator(inv,tot_stake,rb_ratio):
    capital = tot_stake
    excess_profit = 0
    spins = []
    capital_each_spin = []
    for spin in range(1,26):
        capital_each_spin.append(round(capital + excess_profit,2))
        spins.append(spin)
        if capital > 200:    
            excess_profit += capital - 200
            capital = 200
        if capital == 0:
            if excess_profit > 200:
                capital = 200
                excess_profit -= 200
            elif excess_profit > 0:
                capital = excess_profit
                excess_profit = 0
            else:
                break
        rb_stake = (capital)*rb_ratio
        g_stake = capital - 2*rb_stake
        number = random.randint(0,36)
        capital = payoffs(number,rb_stake,g_stake)
        
    if len(capital_each_spin)<25:
        extension = [0]*(25-len(capital_each_spin))
        capital_each_spin = capital_each_spin + extension
        
    return round(capital + excess_profit,2), capital_each_spin

def plot_paths(spins,capital):
    plt.figure(0)
    plt.plot(spins,capital)
    plt.xlabel('Number of spins',size=12)
    plt.ylabel('Capital Remaining',size=12)
    plt.title('Simulated Paths',size=15)
    
def monte_carlo(inv,stake,rb_ratio):
    capital = []
    cap_df = pd.DataFrame(index=list(range(1,26)))
    for spin in range(0,10000):
        remaining_capital, capital_path = simulator(inv,stake,rb_ratio)
        capital.append(remaining_capital)
        cap_df[spin] = capital_path
        
    mean = sum(capital)/len(capital)
    variance = sum((i-mean)**2 for i in capital)/(len(capital)-1)
    sharpe = (mean-investment)/math.sqrt(variance)
    median = statistics.median(capital)
    edge_mc = calculate_edge(capital,inv)
    
    return capital, mean, variance, sharpe, median, cap_df, edge_mc

def calculate_edge(capital,inv):
    unique_returns = list(set(capital))
    edge_dict = {}
    for ret in unique_returns:
        edge_dict[ret] = capital.count(ret)/len(capital)
    
    expected_return = 0
    for key in edge_dict:
        expected_return += edge_dict[key]*key

    edge = ((expected_return-inv)/inv)*100
    return str(round(edge,2)) + "%"

def plot_histogram(capital):
    plt.figure(1)
    plt.hist(capital)
    plt.title('Distribution of Returns',size=15)
    plt.xlabel('Capital Remaining',size=12)
    plt.ylabel('Frequency',size=12)

In [ ]:
ev_edge = expected_value(investment,total_stake)
capital, mean, variance, sharpe, median, cap_df, edge = monte_carlo(investment,total_stake,18/37)

In [ ]:
plot_paths(list(range(1,26)),cap_df)

In [ ]:
plot_histogram(capital)

In [ ]:
edge